<a href="https://colab.research.google.com/github/TetianaHrunyk/NeuralMetaphorsDetection/blob/master/MD1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader
import regex
from time import time

In [34]:
import matplotlib.pyplot as plt

def plot_errors(title, training_losses, test_loss=None, block=True):
    plt.figure(num=title)
    plt.plot(training_losses, label='Training loss')

    if test_loss:
        plt.plot([test_loss]*len(training_losses), label='Test loss')

    plt.tight_layout()
    plt.legend()
    plt.show(block=block)

# Load the data, create custom data set

In [35]:
data_path='https://raw.githubusercontent.com/TetianaHrunyk/NeuralMetaphorsDetection/master/'

In [48]:
for i in range(df.shape[0]):
  if type(df.txt.iloc[i]) != str:
    print(df.txt.iloc[i])

In [36]:
class CustomDataset(Dataset):
  def __init__(self, path, nrows=None, tokenizer=None):
    self.text = pd.read_csv(path, header=None, names=["txt"], dtype='str', nrows=nrows)
    self.text.dropna(inplace=True)
    self.tokenizer = tokenizer

  def __len__(self):
    return self.text.shape[0]

  def __getitem__(self, idx):
    tokenized = self.tokenizer(self.text.txt.iloc[idx])
    labels = " ".join([str(int(word.startswith("M_"))) for word in tokenized])
    txt = " ".join([word.replace("M_", "").lower() for word in tokenized])
    return labels, txt


In [37]:
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

In [38]:
TAGS = ["0", "1"]
train_data = CustomDataset(data_path+"vuamc_train.csv", nrows=None, tokenizer=en_tokenizer)
test_data = CustomDataset(data_path+"vuamc_test.csv",  nrows=100, tokenizer=en_tokenizer)

# Build Vocab

In [39]:
def build_vocab(data):
  counter = Counter()
  for row in data:
      counter.update(row[1].split(" "))
  return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'], min_freq=1)

In [40]:
vocab = build_vocab(train_data)

TypeError: ignored

# Data Loader

In [ ]:
text_pipeline = lambda x: [vocab[token] for token in x.rstrip("\n").split(" ")]
label_pipeline = lambda x: [ int(l) for l in x.split(" ")]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list = [], []
    for labels, text in batch:
        processed_text = text_pipeline(text)
        text_tensor = torch.tensor(processed_text, dtype=torch.int64)
        text_list.append(text_tensor)        
        
        processed_labels=label_pipeline(labels)
        label_tensor = torch.tensor(processed_labels, dtype=torch.int64)
        label_list.append(label_tensor)

    label_list = torch.cat(label_list)
    text_list = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=False, collate_fn=collate_batch)

# Define the model

In [ ]:
class LSTMTagger_Embed(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger_Embed, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)                                                                

        self.hidden2tag =  nn.Linear(hidden_dim, tagset_size) 
        self.hidden = self.init_hidden()         

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim),                                     
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds =  self.word_embeddings(sentence)                                                                                               
        lstm_out, self.hidden =  self.lstm(embeds.view(len(sentence), 1, -1))               
        tag_prediction = self.hidden2tag(lstm_out.view(len(sentence), -1))                     
        tag_scores = F.log_softmax(tag_prediction, dim=1)                       
        return tag_scores

def train(model, n_epochs, dataloader, loss_function, optimizer):
    model.train()
    train_losses = []
    for epoch in range(n_epochs):
        epoch_start = time()
        for i, (tags, sentence) in enumerate(dataloader):
            model.zero_grad()
            model.hidden = model.init_hidden()
            
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            train_losses.append(loss)
            
            loss.backward()
            optimizer.step()
        epoch_end = time()
        print('Epoch %d, loss: %0.4f, epoch took %d sec' % (epoch + 1, loss, round(epoch_end-epoch_start)))
    
    return train_losses

def report_accuracy(model, dataloader, n=None, print_data=False):
    if n==None:
      n=TRAIN_DATA_LEN
    with torch.no_grad():
        total, total_correct, total_exact_correct = 0, 0, 0
        TP, TN, FP, FN = 0, 0, 0, 0
        for tags, sentence in dataloader:
            scores = model(sentence)
            out = torch.argmax(scores, dim=1).tolist()
            targets = tags.tolist()
            
            correct = 0
            length = len(targets)
            
            for i in range(length):
                if out[i] == targets[i]:
                  correct += 1
                  if targets[i] == 1:
                    TP += 1
                  elif targets[i] == 0:
                    TN += 1
                else:
                  if targets[i] == 1:
                    FN += 1
                  elif targets[i] == 0:
                    FP += 1

            if (print_data):
                print('data: ' + str(dict(zip(sentence, tags))))
                print('pred: ' + str(dict(zip(sentence, out))))
                print('Correct: %d of %d' % (correct, length))

            total += length
            total_correct += correct
            if correct == length:
                total_exact_correct += 1

        print('Accuracy (exact): %d / %d, %0.4f' % (total_exact_correct, n, total_exact_correct / n))
        print('Accuracy: %d / %d, %0.4f \n' % (total_correct, total, total_correct / total))
        print("{:5}| {:10} | {:10}\n{}".format("", "T", "F", "-"*30))
        print("{:5}| {:10} | {:10}".format("P", TP, FP))
        print("{:5}| {:10} | {:10}\n".format("N", TN, FN))
        
        den = 0.00001 if TP+FP == 0 else TP+FP
        print("Precision: ", round(TP/den, 2))
        den = 0.00001 if TP+FN == 0 else TP+FN
        print("Recall: ", round(TP/den, 2))
        den = 0.00001 if TP+TN+FP+FN == 0 else TP+TN+FP+FN
        print("F1: ", 2*TP/den)


# Train model

In [ ]:
torch.manual_seed(1)

HIDDEN_DIM = 10  
EMBEDDING_DIM = 64 
n_epochs = 3      
TRAIN_DATA_LEN = len(train_data)
TEST_DATA_LEN = len(test_data)
TAGS = [0, 1]


model = LSTMTagger_Embed(EMBEDDING_DIM, HIDDEN_DIM, len(vocab), len(TAGS))

optimizer = optim.SGD(model.parameters(), lr=0.1)            
loss_function = nn.NLLLoss()      

train_losses = train(model, n_epochs, train_dataloader, loss_function, optimizer)

print("\nTraining Accuracy")
report_accuracy(model, train_dataloader)
plot_errors(training_losses=train_losses, title='Training loss')
#print("\nAccuracy for testing data")
#report_accuracy(model, test_dataloader)
